<a href="https://colab.research.google.com/github/himani26/CaseStudy/blob/main/Trinity_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trinity ChatQnA**:


# **Installations**

In [ ]:

!pip install -q transformers peft datasets bitsandbytes accelerate sentencepiece pypdf2 streamlit pyngrok anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.3 MB/s eta 0:00:00


In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:

import os, torch, textwrap, json, openai
from PyPDF2 import PdfReader
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel

In [ ]:
# 3️⃣ OpenAI API Key (for Q&A generation)
from anthropic import Anthropic
os.environ["ANTHROPIC_API_KEY"] = "secret_key_here"

client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

# **Upload files for training**

In [ ]:

from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving Case Study.pdf to Case Study.pdf


In [ ]:

def extract_text_from_file(path):
    if path.lower().endswith(".pdf"):
        reader = PdfReader(path)
        return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif path.lower().endswith(".txt"):
        return open(path).read()
    else:
        raise ValueError("Only PDF or TXT supported")

doc_text = extract_text_from_file(file_name)
print("✅ Loaded document:", file_name, f"({len(doc_text)} characters)")

✅ Loaded document: Case Study.pdf (3596 characters)


# **Generating QnA Pairs**
For each chunk generating 8 chunks for training purpose

In [ ]:

def generate_qna_pairs(text, num_pairs=8):
    import re, json
    chunks = textwrap.wrap(text, 1500)
    qa_pairs = []


    for chunk in chunks[:3]:
        prompt = f"""
        Generate {num_pairs} factual question–answer pairs from the text below.
        Return ONLY valid JSON array (no markdown, no explanations).

        Text:
        {chunk}
        """
        response = client.messages.create(
            model="claude-3-opus-20240229",
            messages=[

              {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000
        )

        # content = response["choices"][0]["message"]["content"]
        content = response.content[0].text.strip()
        content = re.sub(r"^```json|```$", "", content.strip())
        content = re.sub(r"^```|```$", "", content.strip())

        match = re.search(r"\[.*\]", content, re.S)
        if match:
            json_str = match.group(0)
            try:
                pairs = json.loads(json_str)
                qa_pairs.extend(pairs)
            except Exception as e:
                print("⚠️ JSON decode error:", e)
        else:
            print("⚠️ No valid JSON block detected.")

    return qa_pairs

In [ ]:
lis_qna = generate_qna_pairs(doc_text)

/tmp/ipython-input-1672486378.py:15: DeprecationWarning: The model 'claude-3-opus-20240229' is deprecated and will reach end-of-life on January 5th, 2026.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


In [ ]:
print(lis_qna)

[{'question': 'What is the brand name of Evolocumab?', 'answer': 'Repatha'}, {'question': 'What drug class does Evolocumab belong to?', 'answer': 'PCSK9 inhibitor'}, {'question': 'Who is the manufacturer of Repatha?', 'answer': 'Amgen'}, {'question': 'In what year was Repatha approved by the FDA?', 'answer': '2015'}, {'question': 'What is one of the indications for Repatha usage?', 'answer': 'Reduces risk of myocardial infarction, stroke, and coronary revascularization in adults with established cardiovascular disease.'}, {'question': 'What is a common dosage of Repatha?', 'answer': '140 mg subcutaneously every 2 weeks'}, {'question': 'How is Repatha administered?', 'answer': 'Subcutaneous injection'}, {'question': 'At what temperature should Repatha be stored?', 'answer': 'Refrigerate at 2°C to 8°C (36°F to 46°F)'}, {'question': 'What is the relative risk reduction of CV events in ASCVD patients with Repatha?', 'answer': 'Repatha reduces the relative risk of CV events by 15% in ASCVD 

# **Tokenizing the generated QnA Pairs**

In [ ]:

for item in lis_qna:
    if "question" in item:
        item["instruction"] = item.pop("question")
    if "answer" in item:
        item["response"] = item.pop("answer")
dataset = Dataset.from_list(lis_qna)
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(base_model)

def format_and_tokenize(example):
    prompt = f"<|user|> {example['instruction']}\n<|assistant|> {example['response']}"
    toks = tokenizer(prompt, truncation=True, max_length=512, padding="max_length")
    toks["labels"] = toks["input_ids"].copy()
    return toks

tokenized_ds = dataset.map(format_and_tokenize)
print("✅ Tokenized", len(tokenized_ds), "examples")

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

✅ Tokenized 24 examples


# **Fine Tuning with LORA**

In [ ]:

model = AutoModelForCausalLM.from_pretrained(base_model, load_in_8bit=True, device_map="auto")
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj","v_proj"],
                         lora_dropout=0.1, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=10,
    output_dir="./lora-finetune",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(model=model, train_dataset=tokenized_ds, args=training_args)
print("🚀 Fine-tuning started...")
trainer.train()
model.save_pretrained("lora-finetune")
tokenizer.save_pretrained("lora-finetune")
print("✅ LoRA adapter saved in ./lora-finetune")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


🚀 Fine-tuning started...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


✅ LoRA adapter saved in ./lora-finetune


In [ ]:

%%writefile streamlit_lora_app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

st.set_page_config(page_title="LoRA QA Chatbot", layout="wide")
st.title("💬 LoRA Fine-Tuned Chatbot")

@st.cache_resource
def load_model():
    base = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", load_in_8bit=True, device_map="auto")
    model = PeftModel.from_pretrained(base, "lora-finetune")
    tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    return model, tokenizer

model, tokenizer = load_model()

def generate_answer(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.inference_mode():
        output = model.generate(**input_ids, max_new_tokens=512, temperature=0.7,eos_token_id=None)
    txt = tokenizer.decode(output[0], skip_special_tokens=True)
    # Remove role tokens
    txt = txt.replace("<|user|>", "").replace("<|assistant|>", "").strip()
    return txt.split("### Response:")[-1].strip()

if "chat" not in st.session_state:
    st.session_state.chat = []

user_input = st.text_input("Ask something related to the uploaded document:")
if st.button("Send") and user_input:
    prompt = f"<|user|> {user_input}\n<|assistant|>"
    answer = generate_answer(prompt)
    st.session_state.chat.append(("🧑", user_input))
    st.session_state.chat.append(("🤖", answer))

for s, m in reversed(st.session_state.chat):
    if s == "🧑": st.markdown(f"**{s} You:** {m}")
    else: st.info(f"**{s} Bot:** {m}")

Overwriting streamlit_lora_app.py


In [ ]:

from pyngrok import ngrok
ngrok.set_auth_token("secret_key_here")
print("🚀 Launching chatbot...")
get_ipython().system_raw("streamlit run streamlit_lora_app.py --server.port 8501 &")
public_url = ngrok.connect(8501).public_url
print("✅ Your chatbot is live at:", public_url)

🚀 Launching chatbot...
✅ Your chatbot is live at: https://uncoaxal-nonaccentually-nona.ngrok-free.dev
